# ***File dùng để Upsert các Vector đã được lưu ngoài file JSON lên PINECONE***

In [2]:
# Tải các thư viện cần thiết
import json
from pinecone.grpc import PineconeGRPC as Pinecone
from dotenv import load_dotenv
import os

c:\Workspace\DACNTT\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Workspace\DACNTT\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.29.0 is exactly one major version older than the runtime version 6.31.1 at db_data_2025-01.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(


In [9]:
# Khởi tạo các biến cần thiết
load_dotenv("../.env")

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
HOST_DENSE = os.getenv("HOST_DENSE")
HOST_SPARSE = os.getenv("HOST_SPARSE")

In [18]:
# Đọc các file JSON
#Tạo hàm đọc
def load_chunks_from_json(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        chunks = json.load(f)
    return chunks

In [19]:
# Đọc Dense và Sparse của môn Triết Học
TrietHoc_dense_vector = load_chunks_from_json(r"./TrietHoc/TrietHoc_Dense.json")
TrietHoc_sparse_vector = load_chunks_from_json(r"./TrietHoc/TrietHoc_Sparse.json")

# Đọc Dense và Sparse của môn Lịch Sử Đảng
LSD_dense_vector = load_chunks_from_json(r"./LichSuDang/Lich_Su_Dang_Dense.json")
LSD_sparse_vector = load_chunks_from_json(r"./LichSuDang/Lich_Su_Dang_Sparse.json")

In [8]:
pc = Pinecone(api_key=PINECONE_API_KEY)

# Lấy ra Index của Dense Database
dense_index = pc.Index(host=HOST_DENSE)

# Lấy ra Index của Sparse Database
sparse_index = pc.Index(host=HOST_SPARSE)

## ***Upsert môn Triết Học***

In [21]:
dense_index.upsert(
  vectors = TrietHoc_dense_vector,
  namespace="triet-hoc"
)
sparse_index.upsert(
  vectors = TrietHoc_sparse_vector,
  namespace="triet-hoc"
)

upserted_count: 400

## ***Upsert môn Lịch Sử Đảng***

In [22]:
dense_index.upsert(
  vectors=LSD_dense_vector,
  namespace="lich-su-dang"
)
sparse_index.upsert(
  vectors=LSD_sparse_vector,
  namespace="lich-su-dang"
)

upserted_count: 347

## ***Upsert môn Tư Tưởng Hồ Chí Minh***

In [23]:
## Updating......